# Midterm 2
#### This notebook simulates 1000 days of work in a commercial establishment
#### which recives jobs following a exponential distribution 5/10, and the service
#### time follows a distribution 6/10

In [49]:
#The import block: modules needed are imported here.

from random import expovariate
from numpy import mean

In [50]:
#This block has the function that is intended to calculate the arrival and service time in a random way
#based on the amount of minutes of work in a day.

##It returns two lists: arrival times and services times.

def generateArrivalAndServiceTimes():
    workingDay = 8*60 #8 hours expressed in minutes.
    arrivals = [] #This will contain the arrival times for n works in a day.
    services = [] #This will contain the service times for n works in a day.
    totalAmountOfHours = 0
    while totalAmountOfHours < workingDay:
        nArrivalTime = expovariate(0.5)
        nServiceTime = expovariate(0.6)
        totalAmountOfHours += nArrivalTime
        arrivals.append(nArrivalTime)
        services.append(nServiceTime)
    return arrivals, services

In [51]:
#This blocks contains the delay time function

##It returns two lists: delay time, and departure time.

def delayTime(a, s):
    c = [0.0]*len(a)
    i = 0
    d = [0.0]*len(a)
    while i < len(a)-1:
        i+=1
        if a[i] < c[i-1]:
            d[i] = c[i-1] - a[i]
        else:
            d[i] = 0
        c[i] = a[i] + d[i] + s[i]
    return d, c

In [52]:
#This block contains the job-averaged statistics

def averageInterarrival(a):
    r = mean(a)
    aR = 1/r
    return r, aR

def averageService(s):
    s = mean(s)
    sR = 1/s
    return s, sR

def averageDelay(delay):
    d = mean(delay)
    return d

def averageWait(d, s):
    w = mean(d) + mean(s)
    return w

In [53]:
#This block contains the time-averaged statistics using little's theorem.

#Ew //Sumatory of w divided by thao.
#It recieves three parameters: delay times (list), service times (list), departure times (list).
def little_jobsInServiceNode(d, s, c):
    w = 0.0
    for i, j in zip(d, s):
        w+=(i+j)
    return w/c[-1]

#Ed //Sumatory of d divided by thao
#It recieves two parameters: delay times (list), departure times (list).
def little_jobsInQueue(d, c):
    sumD = 0.0
    for i in d:
        sumD+=i
    return sumD/c[-1]

#Es //Sumatory of s divided by thao
#It recieves two parameters: service times (list), departure times (list).
def little_jobsInService(s, c):
    sumS = 0.0
    for i in s:
        sumS+=i
    return sumS/c[-1]

In [57]:
#This blocks contains the interval estimation function.
#Recieves a list and a number as parameters.
def intervalEstimation(data, confidence):
    xMean = mean(data)
    s2 = 0
    for i in data:
        s2 += (i - xMean)**2  
    s2 = s2/len(data)
    s = s2**(1/2) #Standard deviation
    alpha = (100 - confidence)/100
    t = 1-alpha/2
    
    start = xMean - (t * s)/(len(data)-1)**(1/2)
    end = xMean + (t * s)/(len(data)-1)**(1/2)
    
    interval = {'start': start, 'end': end}
    return interval

In [65]:
amountOfJobs = []

avgInterarrivals = []; arrivalRate = []; avgServices = []; serviceRate = []; avgDelays = []; avgWaits = []

jbsInServiceNode = []; jbsInQueue = []; jbsInService = []
for i in range(1000):
    arrivals, services = generateArrivalAndServiceTimes()
    amountOfJobs.append(len(arrivals))
    delays, c = delayTime(arrivals, services)
    aI, aR = averageInterarrival(arrivals)
    avgInterarrivals.append(aI); arrivalRate.append(aR)
    
    aS, sR = averageService(services)
    avgServices.append(aS); serviceRate.append(sR)
    
    avgDelays.append(averageDelay(delays)); avgWaits.append(averageWait(delays, services))
    
    jbsInServiceNode.append(little_jobsInServiceNode(delays, services, c))
    jbsInQueue.append(little_jobsInQueue(delays, c))
    jbsInService.append(little_jobsInService(services, c))    

print("###########  Results ############\n")
print("#### Amount of jobs intervals: {}\n".format(intervalEstimation(amountOfJobs, 95)))

print("#### Job-average statistics intervals: \n")
print("Average interarrival time intervals: {} | Arrivals rate intervals: {}\n".format(intervalEstimation(avgInterarrivals, 95), intervalEstimation(arrivalRate, 95)))
print("Average service time intervals: {} | Service rate intervals: {}\n".format(intervalEstimation(avgServices, 95), intervalEstimation(serviceRate, 95)))
print("Average delay time intervals: {}\n".format(intervalEstimation(avgDelays, 95)))
print("Average wait time intervals: {}\n".format(intervalEstimation(avgWaits, 95)))

print("#################################\n")
print("#### Time-average statistics intervals: \n")
print("Intervals for time in service node: {}\n".format(intervalEstimation(jbsInServiceNode, 95)))
print("Intervals for time in queue: {}\n".format(intervalEstimation(jbsInQueue, 95)))
print("Intervals for time in service: {}\n".format(intervalEstimation(jbsInService, 95)))


###########  Results ############

#### Amount of jobs intervals: {'start': 240.21770576086863, 'end': 241.17029423913135}

#### Job-average statistics intervals: 

Average interarrival time intervals: {'start': 2.007069200825413, 'end': 2.015134061095877} | Arrivals rate intervals: {'start': 0.4983281954643107, 'end': 0.5003109474525802}

Average service time intervals: {'start': 1.6653657545672735, 'end': 1.672082537013813} | Service rate intervals: {'start': 0.6006052519394789, 'end': 0.6030385203108031}

Average delay time intervals: {'start': 198.6022151162109, 'end': 199.8404383283538}

Average wait time intervals: {'start': 200.2686588972696, 'end': 201.5114428388762}

#################################

#### Time-average statistics intervals: 

Intervals for time in service node: {'start': 119.67141858507622, 'end': 120.2162203636021}

Intervals for time in queue: {'start': 118.67486163906295, 'end': 119.21963284559138}

Intervals for time in service: {'start': 0.996353635896460